In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.getcwd()

'/content'

In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

# path to json file that stores MFCCs and genre labels for each porcessed segment
DATASET_PATH = "/content/drive/MyDrive/DL_for_Audio/data/json/data.json"  

def load_data(dataset_path):
  """Loads training dataset from json file.

      :param data_path (str) : path to json file containing data
      :return X (ndarray) : Inputs
      :return Y (ndarray) : Targets
  """
  
  with open(dataset_path, "r") as fp:
    data = json.load(fp)

  # convert lists into numpy arrays
  X = np.array(data["mfcc"])          # mfcc, labels 값이 list이므로 -> numpy array로 변경
  y = np.array(data["labels"])       

  return X, y

if __name__ == "__main__" :

  # load data
  X, y = load_data(DATASET_PATH)

  # split the data into train and test sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
  
  # build the network architecture
  model = keras.Sequential([
                            
      # input layer
      keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),  # data를 1차원으로 flatten

      # 1st hidden(dense) layer
      keras.layers.Dense(512, activation="relu"),  

      # 2nd hidden layer
      keras.layers.Dense(256, activation="relu"),

      # 3rd hidden layer
      keras.layers.Dense(64, activation="relu"),

      # output layer
      keras.layers.Dense(10, activation="softmax")
  ])

  # compile model(network)
  optimizer = keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(optimizer=optimizer,
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])
  
  model.summary()

  # train model
  history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=50)

  # p